In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd
import geopandas as gpd

In [3]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "2g")
    .config("spark.executer.memory", "4g")
    .getOrCreate()
)

22/08/30 21:09:06 WARN Utils: Your hostname, DESKTOP-80AOBLL resolves to a loopback address: 127.0.1.1; using 172.24.75.21 instead (on interface eth0)
22/08/30 21:09:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/30 21:09:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
merchant_df = pd.read_parquet("../data/tables/tbl_merchants.parquet")
print("Number of merchants in the dataset:", merchant_df.shape[0])
print("Columns in merchant dataset:", [merchant_df.index.name] + merchant_df.columns.tolist())

# extract tags, revenue level and take rate from the "tags" column
merchant_df["tags"] = merchant_df["tags"].str.findall(r'[\(\[]+([^\)\]]*)[\)\]]')
merchant_df["revenue_level"] = merchant_df["tags"].str[1]
merchant_df["take_rate"] = merchant_df["tags"].str[2].str.extract(r'[^\d]*([\d.]*)').astype(float)
merchant_df["tags"] = merchant_df["tags"].str[0].str.lower()  # convert all letters to lowercase
merchant_df

Number of merchants in the dataset: 4026
Columns in merchant dataset: ['merchant_abn', 'name', 'tags']


name  \
merchant_abn                                         
10023283211                          Felis Limited   
10142254217               Arcu Ac Orci Corporation   
10165489824                       Nunc Sed Company   
10187291046   Ultricies Dignissim Lacus Foundation   
10192359162                    Enim Condimentum PC   
...                                            ...   
99938978285                     Elit Dictum Eu Ltd   
99974311662                             Mollis LLP   
99976658299                         Sociosqu Corp.   
99987905597                  Commodo Hendrerit LLC   
99990536339                               Massa PC   

                                                           tags revenue_level  \
merchant_abn                                                                    
10023283211   furniture, home furnishings and equipment shop...             e   
10142254217   cable, satellite, and other pay television and...             b   
10165489824         jewelry, watch, clock, and silverware shops             b   
10187291046              watch, clock, and jewelry repair shops             b   
10192359162   music shops - musical instruments, pianos, and...             a   
...                                                         ...           ...   
99938978285            opticians, optical goods, and eyeglasses             b   
99974311662                  books, periodicals, and newspapers             b   
99976658299                                         shoe  shops             a   
99987905597                motor vehicle supplies and new parts             a   
99990536339                motor vehicle supplies and new parts             e   

              take_rate  
merchant_abn             
10023283211        0.18  
10142254217        4.22  
10165489824        4.40  
10187291046        3.29  
10192359162        6.33  
...                 ...  
99938978285        4.50  
99974311662        3.17  
99976658299        6.57  
99987905597        6.82  
99990536339        0.18  

[4026 rows x 4 columns]

In [5]:
transaction_sdf = spark.read.parquet("../data/tables/transactions_20210228_20210827_snapshot/")
transaction_sdf.count()

3643266

In [6]:
transaction_sdf.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)



In [7]:
transaction_sdf

+-------+------------+------------------+--------------------+--------------+
|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|
+-------+------------+------------------+--------------------+--------------+
|  18478| 62191208634|63.255848959735246|949a63c8-29f7-4ab...|    2021-08-20|
|      2| 15549624934| 130.3505283105634|6a84c3cf-612a-457...|    2021-08-20|
|  18479| 64403598239|120.15860593212783|b10dcc33-e53f-425...|    2021-08-20|
|      3| 60956456424| 136.6785200286976|0f09c5a5-784e-447...|    2021-08-20|
|  18479| 94493496784| 72.96316578355305|f6c78c1a-4600-4c5...|    2021-08-20|
|      3| 76819856970|  448.529684285612|5ace6a24-cdf0-4aa...|    2021-08-20|
|  18479| 67609108741|  86.4040605836911|d0e180f0-cb06-42a...|    2021-08-20|
|      3| 34096466752| 301.5793450525113|6fb1ff48-24bb-4f9...|    2021-08-20|
|  18482| 70501974849| 68.75486276223054|8505fb33-b69a-412...|    2021-08-20|
|      4| 49891706470| 48.89796461900801|ed11e477-b09f-4ae...|    2021-08-20|
|  18482| 46804135891|6.6168976971833615|05b5edb5-b925-414...|    2021-08-20|
|      7| 33064796871| 373.0873675184212|fe188788-b89f-4dd...|    2021-08-20|
|  18483| 44160392990| 83.98473054761176|7c44302d-98b3-48b...|    2021-08-20|
|      7| 68435002949| 232.5364986739752|b4a89891-a113-45e...|    2021-08-20|
|  18484| 70033549200| 871.7678061729196|4b1d941a-21a1-43e...|    2021-08-20|
|      7| 41944909975|30.910755230234322|302ae628-8eba-4a5...|    2021-08-20|
|  18485| 41705715409| 309.5417224575787|67f4f98e-3149-4e8...|    2021-08-20|
|      8| 29566626791| 74.15732460440282|71a81652-cc91-4bf...|    2021-08-20|
|  18487| 32361057556|119.19055863068847|15577921-1104-470...|    2021-08-20|
|      9| 47663262928| 36.69873283148887|c4fcb49a-ce87-4e1...|    2021-08-20|
+-------+------------+------------------+--------------------+--------------+
only showing top 20 rows

In [8]:
consumer_sdf = spark.read.option("delimiter", "|").csv("../data/tables/tbl_consumer.csv", inferSchema =True, header=True)
id_sdf = spark.read.parquet("../data/tables/consumer_user_details.parquet")
for field in ("consumer", "user"):
    field = f'{field}_id'
    id_sdf = id_sdf.withColumn(field, F.col(field).cast('INT'))
output = id_sdf.join(consumer_sdf,["consumer_id"],how='outer')
output.printSchema()

root
 |-- consumer_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postcode: integer (nullable = true)
 |-- gender: string (nullable = true)



In [9]:
# map user id to consumer id
consumer_df = pd.read_csv("../data/tables/tbl_consumer.csv", delimiter="|")
consumer_user_df = pd.read_parquet("../data/tables/consumer_user_details.parquet")
consumer = consumer_user_df.merge(consumer_df, how="outer", on="consumer_id")
consumer

user_id  consumer_id               name  \
0             1      1195503   Yolanda Williams   
1             2       179208         Mary Smith   
2             3      1194530      Jill Jones MD   
3             4       154128    Lindsay Jimenez   
4             5       712975  Rebecca Blanchard   
...         ...          ...                ...   
499994   499995      1385608      Jessica Avila   
499995   499996      1466964    Steven Thornton   
499996   499997      1253484      Christy Smith   
499997   499998       175005       Donna Sutton   
499998   499999       746835     Hannah Wilkins   

                                address state  postcode       gender  
0            413 Haney Gardens Apt. 742    WA      6935       Female  
1                       3764 Amber Oval   NSW      2782       Female  
2                    40693 Henry Greens    NT       862       Female  
3             00653 Davenport Crossroad   NSW      2780       Female  
4         9271 Michael Manors Suite 651    WA      6355       Female  
...                                 ...   ...       ...          ...  
499994    508 Miranda Overpass Apt. 218   QLD      4400       Female  
499995  7913 Schwartz Mission Suite 483   VIC      3097  Undisclosed  
499996   5681 Zachary Mountain Apt. 060   NSW      2756  Undisclosed  
499997                54140 Jacob Point   VIC      3989       Female  
499998                61055 Long Valley   NSW      1755       Female  

[499999 rows x 7 columns]

In [10]:
consumer.groupby("postcode")["postcode"].count().reset_index(name="count")

postcode  count
0          200    145
1          800    155
2          801    147
3          804    156
4          810    150
...        ...    ...
3162      9013    137
3163      9015    136
3164      9464    165
3165      9726    178
3166      9999    149

[3167 rows x 2 columns]

In [16]:
sf = gpd.read_file("../data/external/SA2_2021/SA2_2021_AUST_GDA2020.shp")
sf = sf.set_index('SA2_CODE21')
sf = sf.loc[sf.geometry != None]
sf

SA2_NAME21 CHG_FLAG21  CHG_LBL21 SA3_CODE21  \
SA2_CODE21                                                                     
101021007                         Braidwood          0  No change      10102   
101021008                           Karabar          0  No change      10102   
101021009                        Queanbeyan          0  No change      10102   
101021010                 Queanbeyan - East          0  No change      10102   
101021012   Queanbeyan West - Jerrabomberra          0  No change      10102   
...                                     ...        ...        ...        ...   
801111141                           Namadgi          0  No change      80111   
901011001                  Christmas Island          0  No change      90101   
901021002           Cocos (Keeling) Islands          0  No change      90102   
901031003                        Jervis Bay          0  No change      90103   
901041004                    Norfolk Island          0  No change      90104   

                         SA3_NAME21 SA4_CODE21                    SA4_NAME21  \
SA2_CODE21                                                                     
101021007                Queanbeyan        101                Capital Region   
101021008                Queanbeyan        101                Capital Region   
101021009                Queanbeyan        101                Capital Region   
101021010                Queanbeyan        101                Capital Region   
101021012                Queanbeyan        101                Capital Region   
...                             ...        ...                           ...   
801111141         Uriarra - Namadgi        801  Australian Capital Territory   
901011001          Christmas Island        901             Other Territories   
901021002   Cocos (Keeling) Islands        901             Other Territories   
901031003                Jervis Bay        901             Other Territories   
901041004            Norfolk Island        901             Other Territories   

           GCC_CODE21                    GCC_NAME21 STE_CODE21  \
SA2_CODE21                                                       
101021007       1RNSW                   Rest of NSW          1   
101021008       1RNSW                   Rest of NSW          1   
101021009       1RNSW                   Rest of NSW          1   
101021010       1RNSW                   Rest of NSW          1   
101021012       1RNSW                   Rest of NSW          1   
...               ...                           ...        ...   
801111141       8ACTE  Australian Capital Territory          8   
901011001       9OTER             Other Territories          9   
901021002       9OTER             Other Territories          9   
901031003       9OTER             Other Territories          9   
901041004       9OTER             Other Territories          9   

                              STE_NAME21 AUS_CODE21 AUS_NAME21  AREASQKM21  \
SA2_CODE21                                                                   
101021007                New South Wales        AUS  Australia   3418.3525   
101021008                New South Wales        AUS  Australia      6.9825   
101021009                New South Wales        AUS  Australia      4.7620   
101021010                New South Wales        AUS  Australia     13.0032   
101021012                New South Wales        AUS  Australia     13.6748   
...                                  ...        ...        ...         ...   
801111141   Australian Capital Territory        AUS  Australia   1202.7527   
901011001              Other Territories        AUS  Australia    136.1356   
901021002              Other Territories        AUS  Australia     13.7258   
901031003              Other Territories        AUS  Australia     67.2296   
901041004              Other Territories        AUS  Australia     38.6510   

                                                   LOCI_URI21  \
SA2_CODE21  

In [11]:
# sf_poa = gpd.read_file("../data/curated/POA_2021/POA_2021_AUST_GDA2020.shp")

# sf_poa = sf_poa.loc[sf_poa["geometry"] != None]
# sf_poa['postcode'] = sf_poa['POA_CODE21'].astype(int)
# sf_poa.head()

In [19]:
# link: https://www.matthewproctor.com/Content/postcodes/australian_postcodes.csv
all_postcode = pd.read_csv("../data/external/australian_postcodes.csv")
all_postcode

id  postcode                        locality state        long  \
0        230       200                             ANU   ACT  149.119000   
1      21820       200  Australian National University   ACT  149.118900   
2        232       800                          DARWIN    NT  130.836680   
3      24049       800                     DARWIN CITY    NT  130.836680   
4        233       801                          DARWIN    NT  130.836680   
...      ...       ...                             ...   ...         ...   
18437  11186      9013                        BRISBANE   QLD  152.823141   
18438  11187      9015                        BRISBANE   QLD  152.823141   
18439  11196      9464                    NORTHGATE MC   QLD  153.074982   
18440  11197      9726                   GOLD COAST MC   QLD  153.412197   
18441  23878      9999                      NORTH POLE   VIC  144.956776   

             lat                    dc           type  \
0     -35.277700                   NaN            NaN   
1     -35.277700                   NaN            NaN   
2     -12.458684                   NaN            NaN   
3     -12.458684                   NaN            NaN   
4     -12.458684                   NaN            NaN   
...          ...                   ...            ...   
18437 -27.603479    CITY DC - BRISBANE            LVR   
18438 -27.603479    CITY DC - BRISBANE            LVR   
18439 -27.397055                   NaN            NaN   
18440 -28.008783                   NaN            NaN   
18441 -37.817403  CITY DELIVERY CENTRE  Delivery Area   

                        status      sa3  ... MMM_2015  MMM_2019        ced  \
0                          NaN      NaN  ...      1.0       1.0        NaN   
1            Added 19-Jan-2020      NaN  ...      1.0       1.0        NaN   
2           Updated 6-Feb-2020  70101.0  ...      2.0       2.0        NaN   
3           Updated 6-Feb-2020  70101.0  ...      2.0       2.0        NaN   
4      Updated 25-Mar-2020 SA3  70101.0  ...      2.0       2.0        NaN   
...                        ...      ...  ...      ...       ...        ...   
18437  Updated 25-Mar-2020 SA3  30504.0  ...      1.0       1.0        NaN   
18438  Updated 25-Mar-2020 SA3  30504.0  ...      1.0       1.0        NaN   
18439  Updated 25-Mar-2020 SA3  30203.0  ...      1.0       1.0        NaN   
18440  Updated 25-Mar-2020 SA3  30910.0  ...      1.0       1.0  Moncrieff   
18441        Added 1-July-2021  20604.0  ...      1.0       1.0  Melbourne   

        altitude  chargezone  phn_code            phn_name   lgaregion  \
0            NaN          N2       NaN                 NaN         NaN   
1            NaN          N2       NaN                 NaN         NaN   
2            NaN         NT1    PHN701  Northern Territory      Darwin   
3            NaN         NT1    PHN701  Northern Territory      Darwin   
4            NaN         NT1    PHN701                 NaN         NaN   
...          ...         ...       ...                 ...         ...   
18437  44.349792          Q1    PHN301                 NaN    Brisbane   
18438  44.349792          Q1    PHN301                 NaN    Brisbane   
18439        NaN          Q1    PHN301                 NaN    Brisbane   
18440        NaN          Q1    PHN303                 NaN  Gold Coast   
18441  15.244751         NaN       NaN                 NaN       Yarra   

       electorate    electoraterating  
0          Durack                 NaN  
1          Durack                 NaN  
2         Solomon  Inner Metropolitan  
3         Solomon  Inner Metropolitan  
4        Lingiari               Rural  
...           ...                 ...  
18437    Griffith                 NaN  
18438    Griffith                 NaN  
18439    Griffith                 NaN  
18440   McPherson                 NaN  
18441   Macnamara                 NaN  

[18442 rows x 36 columns]

In [87]:
all_postcode = all_postcode[['postcode', 'long', 'lat']]
all_postcode

postcode        long        lat
0           200  149.119000 -35.277700
1           200  149.118900 -35.277700
2           800  130.836680 -12.458684
3           800  130.836680 -12.458684
4           801  130.836680 -12.458684
...         ...         ...        ...
18437      9013  152.823141 -27.603479
18438      9015  152.823141 -27.603479
18439      9464  153.074982 -27.397055
18440      9726  153.412197 -28.008783
18441      9999  144.956776 -37.817403

[18442 rows x 3 columns]

In [20]:
all_postcode = all_postcode.drop_duplicates(subset='postcode', keep="first")
all_postcode

id  postcode       locality state        long        lat  \
0        230       200            ANU   ACT  149.119000 -35.277700   
2        232       800         DARWIN    NT  130.836680 -12.458684   
4        233       801         DARWIN    NT  130.836680 -12.458684   
5        234       804          PARAP    NT  130.873315 -12.428017   
6        235       810          ALAWA    NT  130.866242 -12.381806   
...      ...       ...            ...   ...         ...        ...   
18437  11186      9013       BRISBANE   QLD  152.823141 -27.603479   
18438  11187      9015       BRISBANE   QLD  152.823141 -27.603479   
18439  11196      9464   NORTHGATE MC   QLD  153.074982 -27.397055   
18440  11197      9726  GOLD COAST MC   QLD  153.412197 -28.008783   
18441  23878      9999     NORTH POLE   VIC  144.956776 -37.817403   

                         dc           type                   status      sa3  \
0                       NaN            NaN                      NaN      NaN   
2                       NaN            NaN       Updated 6-Feb-2020  70101.0   
4                       NaN            NaN  Updated 25-Mar-2020 SA3  70101.0   
5                       NaN            NaN  Updated 25-Mar-2020 SA3  70102.0   
6                       NaN            NaN       Updated 6-Feb-2020  70102.0   
...                     ...            ...                      ...      ...   
18437    CITY DC - BRISBANE            LVR  Updated 25-Mar-2020 SA3  30504.0   
18438    CITY DC - BRISBANE            LVR  Updated 25-Mar-2020 SA3  30504.0   
18439                   NaN            NaN  Updated 25-Mar-2020 SA3  30203.0   
18440                   NaN            NaN  Updated 25-Mar-2020 SA3  30910.0   
18441  CITY DELIVERY CENTRE  Delivery Area        Added 1-July-2021  20604.0   

       ... MMM_2015  MMM_2019        ced   altitude  chargezone  phn_code  \
0      ...      1.0       1.0        NaN        NaN          N2       NaN   
2      ...      2.0       2.0        NaN        NaN         NT1    PHN701   
4      ...      2.0       2.0        NaN        NaN         NT1    PHN701   
5      ...      2.0       2.0        NaN        NaN         NT1    PHN701   
6      ...      2.0       2.0        NaN   9.221429         NT1    PHN701   
...    ...      ...       ...        ...        ...         ...       ...   
18437  ...      1.0       1.0        NaN  44.349792          Q1    PHN301   
18438  ...      1.0       1.0        NaN  44.349792          Q1    PHN301   
18439  ...      1.0       1.0        NaN        NaN          Q1    PHN301   
18440  ...      1.0       1.0  Moncrieff        NaN          Q1    PHN303   
18441  ...      1.0       1.0  Melbourne  15.244751         NaN       NaN   

                 phn_name   lgaregion  electorate    electoraterating  
0                     NaN         NaN      Durack                 NaN  
2      Northern Territory      Darwin     Solomon  Inner Metropolitan  
4                     NaN         NaN    Lingiari               Rural  
5                     NaN         NaN      Durack                 NaN  
6      Northern Territory      Darwin     Solomon  Inner Metropolitan  
...                   ...         ...         ...                 ...  
18437                 NaN    Brisbane    Griffith                 NaN  
18438                 NaN    Brisbane    Griffith                 NaN  
18439                 NaN    Brisbane    Griffith                 NaN  
18440                 NaN  Gold Coast   McPherson                 NaN  
18441                 NaN       Yarra   Macnamara                 NaN  

[3167 rows x 36 columns]

In [21]:
from shapely.geometry import Point, Polygon

all_postcode["coordinate"] = all_postcode.apply(lambda x: Point(x.long, x.lat), axis=1)
all_postcode

/home/jinxuan/.local/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/tmp/ipykernel_132/2144914703.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_postcode["coordinate"] = all_postcode.apply(lambda x: Point(x.long, x.lat), axis=1)


id  postcode       locality state        long        lat  \
0        230       200            ANU   ACT  149.119000 -35.277700   
2        232       800         DARWIN    NT  130.836680 -12.458684   
4        233       801         DARWIN    NT  130.836680 -12.458684   
5        234       804          PARAP    NT  130.873315 -12.428017   
6        235       810          ALAWA    NT  130.866242 -12.381806   
...      ...       ...            ...   ...         ...        ...   
18437  11186      9013       BRISBANE   QLD  152.823141 -27.603479   
18438  11187      9015       BRISBANE   QLD  152.823141 -27.603479   
18439  11196      9464   NORTHGATE MC   QLD  153.074982 -27.397055   
18440  11197      9726  GOLD COAST MC   QLD  153.412197 -28.008783   
18441  23878      9999     NORTH POLE   VIC  144.956776 -37.817403   

                         dc           type                   status      sa3  \
0                       NaN            NaN                      NaN      NaN   
2                       NaN            NaN       Updated 6-Feb-2020  70101.0   
4                       NaN            NaN  Updated 25-Mar-2020 SA3  70101.0   
5                       NaN            NaN  Updated 25-Mar-2020 SA3  70102.0   
6                       NaN            NaN       Updated 6-Feb-2020  70102.0   
...                     ...            ...                      ...      ...   
18437    CITY DC - BRISBANE            LVR  Updated 25-Mar-2020 SA3  30504.0   
18438    CITY DC - BRISBANE            LVR  Updated 25-Mar-2020 SA3  30504.0   
18439                   NaN            NaN  Updated 25-Mar-2020 SA3  30203.0   
18440                   NaN            NaN  Updated 25-Mar-2020 SA3  30910.0   
18441  CITY DELIVERY CENTRE  Delivery Area        Added 1-July-2021  20604.0   

       ... MMM_2019        ced   altitude chargezone  phn_code  \
0      ...      1.0        NaN        NaN         N2       NaN   
2      ...      2.0        NaN        NaN        NT1    PHN701   
4      ...      2.0        NaN        NaN        NT1    PHN701   
5      ...      2.0        NaN        NaN        NT1    PHN701   
6      ...      2.0        NaN   9.221429        NT1    PHN701   
...    ...      ...        ...        ...        ...       ...   
18437  ...      1.0        NaN  44.349792         Q1    PHN301   
18438  ...      1.0        NaN  44.349792         Q1    PHN301   
18439  ...      1.0        NaN        NaN         Q1    PHN301   
18440  ...      1.0  Moncrieff        NaN         Q1    PHN303   
18441  ...      1.0  Melbourne  15.244751        NaN       NaN   

                 phn_name   lgaregion  electorate    electoraterating  \
0                     NaN         NaN      Durack                 NaN   
2      Northern Territory      Darwin     Solomon  Inner Metropolitan   
4                     NaN         NaN    Lingiari               Rural   
5                     NaN         NaN      Durack                 NaN   
6      Northern Territory      Darwin     Solomon  Inner Metropolitan   
...                   ...         ...         ...                 ...   
18437                 NaN    Brisbane    Griffith                 NaN   
18438                 NaN    Brisbane    Griffith                 NaN   
18439                 NaN    Brisbane    Griffith                 NaN   
18440                 NaN  Gold Coast   McPherson                 NaN   
18441                 NaN       Yarra   Macnamara                 NaN   

                          coordinate  
0           POINT (149.119 -35.2777)  
2       POINT (130.83668 -12.458684)  
4       POINT (130.83668 -12.458684)  
5      POINT (130.873315 -12.428017)  
6      POINT (130.866242 -12.381806)  
...                              ...  
18437  POINT (152.823141 -27.603479)  
18438  POINT (152.823141 -27.603479)  
18439  POINT (153.074982 -27.397055)  
18440  POINT (153.412197 -28.008783)  
18441  POINT (144.956776 -37.817403)  

[3167 rows x 37 columns]

In [22]:
def postcode_to_SA2(coordinate):
    for SA2_code, row in sf.iterrows():
        if coordinate.within(row['geometry']):
            return SA2_code
            
all_postcode["SA2_code"] = all_postcode.apply(lambda x: postcode_to_SA2(x.coordinate), axis=1)
all_postcode

KeyboardInterrupt: 

In [ ]:
all_postcode.to_csv("../data/curated/processed_postcode.csv", index=False)